In [ ]:
import pandas as pd
import os
bucket =os.getenv('WORKSPACE_BUCKET')

In [ ]:
#get sample file for plink filtering
sam = pd.read_csv(f'{bucket}/data/gwas_v2/phenotypes/celiac_demo_v2.tsv',sep='\t')
sam['person_id'].to_csv('samples_v2.txt',header=None)

In [ ]:
%%bash
#plink files to use for imputation
!gsutil cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/exome_v7.1/plink_bed/exome.chr6*
!gsutil cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr6*



#need to get the genetic data from plink files, combine exome and acaf files
plink --bfile exome.chr6 \
  --keep samples_v2.txt \
  --make-bed \
  --out chr6_filtered_ex \

 plink  --bfile acaf_threshold.chr6 \
  --keep samples_v2.txt \
  --make-bed \
  --out chr6_filtered_acaf \

plink  --bfile chr6_filtered_acaf \
  --bmerge chr6_filtered_ex \
  --make-bed \
  --memory 60000 \
  --out chr6_total \

In [ ]:
from pyliftover import LiftOver
#liftover for genes
lo = LiftOver('hg19','hg38')
for gene in ['A','B','C','DPB1','DQA1','DQB1','DRB1']:
    hg19 = pd.read_csv(f'hg19_{gene}_pos.txt',header=None)
    lifted=[lo.convert_coordinate('chr6', i, '-') for i in hg19[0]]
    hg19['hg38']=[i[0][1] for i in lifted]
    hg19['hg38'].to_csv(f'hg38_{gene}_pos.txt',header=None,index=False)